# Домашнее задание 1

In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
data = pd.read_csv('weatherHistory.csv')

In [ ]:
data.columns

Index(['Formatted Date', 'Summary', 'Precip Type', 'Temperature (C)',
       'Apparent Temperature (C)', 'Humidity', 'Wind Speed (km/h)',
       'Wind Bearing (degrees)', 'Visibility (km)', 'Loud Cover',
       'Pressure (millibars)', 'Daily Summary'],
      dtype='object')

In [ ]:
def clean_column(column):
    column = column.lower()
    column = [word for word in column.split() if not'(' in word]
    column = '_'.join(column)
    return column

In [ ]:
replace = {}
for column in data.columns:
    replace[column] = clean_column(column)
data.rename(replace, axis=1, inplace=True)

In [ ]:
data.rename({'formatted_date': 'date'}, axis=1, inplace=True)

In [ ]:
data.columns

Index(['date', 'summary', 'precip_type', 'temperature', 'apparent_temperature',
       'humidity', 'wind_speed', 'wind_bearing', 'visibility', 'loud_cover',
       'pressure', 'daily_summary'],
      dtype='object')

In [ ]:
data.date = pd.to_datetime(data.date, utc=True)

In [ ]:
data.date

0       2006-03-31 22:00:00+00:00
1       2006-03-31 23:00:00+00:00
2       2006-04-01 00:00:00+00:00
3       2006-04-01 01:00:00+00:00
4       2006-04-01 02:00:00+00:00
                   ...           
96448   2016-09-09 17:00:00+00:00
96449   2016-09-09 18:00:00+00:00
96450   2016-09-09 19:00:00+00:00
96451   2016-09-09 20:00:00+00:00
96452   2016-09-09 21:00:00+00:00
Name: date, Length: 96453, dtype: datetime64[ns, UTC]

In [ ]:
data.sort_values(by='date', inplace=True)

In [ ]:
data.head(4)

,date,summary,precip_type,temperature,apparent_temperature,humidity,wind_speed,wind_bearing,visibility,loud_cover,pressure,daily_summary
2880,2005-12-31 23:00:00+00:00,Partly Cloudy,rain,0.577778,-4.050000,0.89,17.1143,140.0,9.9820,0.0,1016.66,Mostly cloudy throughout the day.
2881,2006-01-01 00:00:00+00:00,Mostly Cloudy,rain,1.161111,-3.238889,0.85,16.6152,139.0,9.9015,0.0,1016.15,Mostly cloudy throughout the day.
2882,2006-01-01 01:00:00+00:00,Mostly Cloudy,rain,1.666667,-3.155556,0.82,20.2538,140.0,9.9015,0.0,1015.87,Mostly cloudy throughout the day.
2883,2006-01-01 02:00:00+00:00,Overcast,rain,1.711111,-2.194444,0.82,14.4900,140.0,9.9015,0.0,1015.56,Mostly cloudy throughout the day.


**1) самый ветрянный месяц**

In [ ]:
by_month = data.groupby(by=data.date.dt.month)['date', 'wind_speed'].mean()

In [ ]:
by_month.loc[by_month.wind_speed == by_month.wind_speed.max()]

,wind_speed
date,
3,13.415825


**2)самый влажный месяц**

In [ ]:
by_year = data.groupby(by=data.date.dt.year)['date', 'humidity'].mean()

In [ ]:
by_year.loc[by_year.humidity == by_year.humidity.max()]

,humidity
date,
2005,0.89


**3) неделя с самой переменчивой погодой**

In [ ]:
by_week = data.groupby(by=data.date.dt.week)['date', 'temperature', 'humidity', 'wind_speed'].std() #данные больше всего отклоняются от среднего

In [ ]:
by_week.sort_values(by=['humidity', 'temperature', 'wind_speed'], ascending=False).head(1)

,temperature,humidity,wind_speed
date,,,
34,6.149509,0.223476,6.005965


# Домашнее задание 2

In [ ]:
import nltk
nltk.download('punkt')
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
data = pd.read_csv('data.csv')

Можно предположить, что объявления с большим количеством предложений более содержательны и, как следствие, более привлекательны для покупателя.

In [ ]:
def number_of_sent(text):
    return len(nltk.tokenize.sent_tokenize(text, language='russian'))

In [ ]:
data['number_of_sent'] = data.description.apply(number_of_sent)

Также предположительно, если в тексте много латинских букв, то значит, что в объявлении указано больше технических характеристик и поэтому оно более привлекательно для покупателя.

In [ ]:
def latin(text):
    return sum([1 for ch in text if ch in string.ascii_letters])

In [ ]:
data['latin'] = data.description.apply(latin)

Если в тексте количество заглавных букв превышает количество строчных, то, возможно, объявление будет восприниматься негативно

In [ ]:
def case(text):
    text = nltk.tokenize.word_tokenize(text, language='russian')
    text = [word for word in text if word.isalpha()]
    upper_case = [word for word in text if word.isupper()]
    lower_case = [word for word in text if word.islower()]
    if not text or len(lower_case) == 0:
        return 0
    return len(upper_case)/len(lower_case)


In [ ]:
data['case'] = data.description.apply(case)

Можно утверждать, что текст с более длинными словами является более содержательным

In [ ]:
def word_len(text):
    text = nltk.tokenize.word_tokenize(text, language='russian')
    text = [len(word) for word in text if word.isalpha()]
    if not text:
        return 0
    return sum(text)/len(text)

In [ ]:
data['word_len'] = data.description.apply(word_len)

In [ ]:
data.corr()

,price,item_seq_number,image_top_1,deal_probability,number_of_sent,latin,case,word_len
price,1.000000,0.077973,0.046540,-0.015032,0.070424,-0.005198,-0.003138,0.029154
item_seq_number,0.077973,1.000000,0.093411,-0.038196,0.099171,0.018651,0.013127,0.039033
image_top_1,0.046540,0.093411,1.000000,0.185397,0.087741,0.176032,0.043833,0.085567
deal_probability,-0.015032,-0.038196,0.185397,1.000000,0.004417,-0.005189,0.000854,0.041363
number_of_sent,0.070424,0.099171,0.087741,0.004417,1.000000,0.116022,0.028059,0.061888
latin,-0.005198,0.018651,0.176032,-0.005189,0.116022,1.000000,0.112506,0.018789
case,-0.003138,0.013127,0.043833,0.000854,0.028059,0.112506,1.000000,-0.025580
word_len,0.029154,0.039033,0.085567,0.041363,0.061888,0.018789,-0.025580,1.000000


С вероятностью покупки коррелирует только длинна слов.

**Векторизация**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import r2_score, mean_squared_error
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
X = data[['number_of_sent', 'latin', 'case', 'word_len']].values
y = data.deal_probability.values

In [ ]:
clf = LinearRegression().fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
y_pred

array([0.15364614, 0.13550428, 0.14701031, ..., 0.139063  , 0.14333984,
       0.14880856])

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred))

0.26338540482969625

In [ ]:
def eval_result(y_test, y_pred):
    print(f'{np.sqrt(mean_squared_error(y_test, y_pred))}')
    print(f'{r2_score(y_test, y_pred)}')

In [ ]:
def evaluation_model(vec):
    X = vec.fit_transform(data.description)
    y = data.deal_probability.values
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    regressor = Ridge(alpha=1).fit(X_train, y_train)
    y_pred = regressor.predict(X_test)
    eval_result(y_test, y_pred)

**Параметры векторизатора CountVectorizer**

In [ ]:
cv = CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,3),
                     stop_words=stopwords.words('russian'))
evaluation_model(cv)

0.2514549837527481
0.09054043348263452


Использование стоп-слов не улучшило существенно результат.

In [ ]:
cv = CountVectorizer(max_features=7000, min_df=5, max_df=0.4, ngram_range=(1,3),
                     stop_words=stopwords.words('russian'))
evaluation_model(cv)

0.25520325386494447
0.06322495021503449


Повышение количества фич не улучшае результат.

In [ ]:
cv = CountVectorizer(max_features=1000, min_df=2, max_df=0.4, ngram_range=(1,3),
                     stop_words=stopwords.words('russian'))
evaluation_model(cv)

0.251459366499311
0.09050873026907102


Уменьшение параметра минимальной частоты не улучшило результат

In [ ]:
cv = CountVectorizer(max_features=1000, min_df=5, max_df=0.4, ngram_range=(1,2),
                     stop_words=stopwords.words('russian'))
evaluation_model(cv)

0.25142451296357105
0.0907608329455104


Если убрать триграммы, то качество немного улучшается

**Параметры векторизатора TfidfVectorizer**

In [ ]:
tf = TfidfVectorizer(max_features=1000, min_df=3, max_df=0.4, stop_words=stopwords.words('russian'))
evaluation_model(tf)

0.24834101734336886
0.11292607789438358


Использование в модели tfidf стоп-слов показывает лучший результат, чем в countvectorizer

In [ ]:
tf = TfidfVectorizer(max_features=7000, min_df=5, max_df=0.4, stop_words=stopwords.words('russian'))
evaluation_model(tf)

0.2477304343643191
0.1172827194387881


Увелечение фич улучшило результат.

In [ ]:
tf = TfidfVectorizer(max_features=7000, min_df=3, max_df=0.8, stop_words=stopwords.words('russian'))
evaluation_model(tf)

0.24774395146412548
0.1171863880927102


При увелечении параметра максимальной частоты качество немного уменьшилось.